In [21]:
import math
from prettytable import PrettyTable
import time

In [22]:
def rigorous2PL(dirty_shedule):
    read_locks = {}
    write_locks = {}
    shedule = cleanShedule(dirty_shedule)
    initiateLocks(read_locks, write_locks, shedule)
    done = False
    actived_transactions = activateAllTransactions(shedule)
    log = []
    
    x = PrettyTable()
    x.field_names = getTransactions(shedule)
    
    t0 = time.time()
    
    while not done:
        for i in shedule:
            op = i[0]
            trans = i[1]
            if "c" not in i:
                element = i[2]
            
            if actived_transactions[int(trans)] == True and i not in log:
                
                if op == "r":
                    
                    if write_locks[element] == 0 or write_locks[element] == int(trans):
                        
                        addReadLock(read_locks, element, trans)
                        rl = op + "l" + trans + "(" + element + ")"
                        r = op + trans + "(" + element + ")"
                        x.add_row(rowArray(trans, rl, shedule))
                        x.add_row(rowArray(trans, r, shedule))
                        log.append(i)
                    else:
                        
                        trylock = "Trylock" + trans + "(" + element + ")"
                        x.add_row(rowArray(trans, trylock, shedule))
                        actived_transactions[int(trans)] = False
                elif op == "w":
                    
                    index = int(trans) - 1
                    if ((write_locks[element] == 0 or write_locks[element] == int(trans)) and 
                        (checkReadLock(read_locks, trans, element))):
        
                        write_locks[element] = int(trans)
                        wl = op + "l" + trans + "(" + element + ")"
                        w = op + trans + "(" + element + ")"
                        x.add_row(rowArray(trans, wl, shedule))
                        x.add_row(rowArray(trans, w, shedule))
                        log.append(i)
                    else:
                        
                        trylock = "Trylock" + trans + "(" + element + ")"
                        x.add_row(rowArray(trans, trylock, shedule))
                        actived_transactions[int(trans)] = False
                elif op == "c":
                    
                    commit = op + trans + "; unlock(" 
                    for lock in unlock(trans, write_locks, read_locks):
                        commit += lock + ", " 
                    commit = commit[:-2] 
                    commit += ")"
                    
                    x.add_row(rowArray(trans, commit, shedule))
                    log.append(i)
                   
                    
                    for num in actived_transactions:
                        if num == int(trans):
                            actived_transactions[num] = None
                        else:
                            if actived_transactions[num] != None:
                                actived_transactions[num] = True
                    
                    
                    if len(log) == len(shedule):
                        done = True
                        
                    break
                    
        if tooLate(t0):
            print("Potential deadlock!")
            break
        
    print(x)
                    
    

def cleanShedule(dirty_shedule):
    shedule = dirty_shedule
    shedule = shedule.replace("(", "").replace(")", "").replace(" ", "")
    if ";" in shedule[-1]:
        shedule = shedule[:-1]
    shedule = shedule.split(";")
    return shedule

def getTransactions(shedule):
    transactions = [] 
    for i in shedule:
        if 'c' not in i:
            trans_num = i[1]
            if ("T" + trans_num) not in transactions:
                transaction = "T" + trans_num
                transactions.append(transaction)
    
    return transactions

def activateAllTransactions(shedule):
    trans_num = len(getTransactions(shedule))
    activated_trans = {}
    for i in range(1, trans_num + 1):
        activated_trans[i] = True
    return activated_trans

def rowArray(trans, op, shedule):
    row = []
    for i in getTransactions(shedule):
        row.append("")
        
    index = int(trans) - 1
    row[index] = op
    
    return row

def getElements(shedule):
    elements = []
    for i in shedule:
        if "c" not in i:
            elements.append(i[2])
    elements = list(set(elements))
    
    return elements

def addReadLock(read_locks, element, trans):
    arr = read_locks[element]
    index = int(trans) - 1
    arr[index] = int(trans)
    read_locks[element] = arr
    
def initiateLocks(read_locks, write_locks, shedule):
    elements = getElements(shedule)
    trans_num = len(getTransactions(shedule))
    for e in elements:
        write_locks[e] = 0
        read = []
        for i in range(trans_num):
            read.append(0)
        read_locks[e] = read
        
def unlock(trans, write_locks, read_locks):
    all_unlocked = []
    index = int(trans) - 1
    for key in write_locks:
        if write_locks[key] == int(trans):
            write_locks[key] = 0
            all_unlocked.append(key)
            
    for key in read_locks:
        a = read_locks[key]
        if a[index] != 0:
            all_unlocked.append(key)
        a[index] = 0
        read_locks[key] = a
    
    all_unlocked = list(set(all_unlocked))
    
    return sorted(all_unlocked)

def checkReadLock(read_locks, trans, element):
    index = int(trans) - 1
    
    for key in read_locks:
        if key == element:
            check = read_locks[key]
            for i in range(len(check)):
                if i != index:
                    if check[i] != 0:
                        return False
    return True

def tooLate(t0):
    t1 = time.time()
    dif = round(t1 - t0)
    if dif >= 3:
        return True
    
    return False

In [23]:
#SKRIV HISTORIE HER 

#skriv i det formatet som står under, ofte er det bare samme format som gis på eksamen;)

historie = "r1(A); w2(B); r1(B); r3(A); w3(A); c1; c2; c3;"

rigorous2PL(historie)

+------------------+---------------+---------------+
|        T1        |       T2      |       T3      |
+------------------+---------------+---------------+
|      rl1(A)      |               |               |
|      r1(A)       |               |               |
|                  |     wl2(B)    |               |
|                  |     w2(B)     |               |
|   Trylock1(B)    |               |               |
|                  |               |     rl3(A)    |
|                  |               |     r3(A)     |
|                  |               |  Trylock3(A)  |
|                  | c2; unlock(B) |               |
|      rl1(B)      |               |               |
|      r1(B)       |               |               |
|                  |               |  Trylock3(A)  |
| c1; unlock(A, B) |               |               |
|                  |               |     wl3(A)    |
|                  |               |     w3(A)     |
|                  |               | c3; unloc